In [2]:
import pandas as pd
import seaborn as snc
import numpy as np
import matplotlib as matplot

In [3]:
df = pd.read_excel('sales.xlsx')
df.head()

,InvoiceNumber,ProductCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### Step 1 – Filter rows with missing Customer IDs
Removed entries with missing `CustomerId` to prepare the data for customer analysis.


In [4]:
df_no_missing_CustomerId = df.copy()

df_no_missing_CustomerId.dropna(subset=['CustomerId'],how='all',inplace=True)

df_no_missing_CustomerId.head()

,InvoiceNumber,ProductCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### Step 2 – Remove fully duplicated rows
Keet only one instance of rows that were completely duplicated


In [5]:
df_no_duplicate = df_no_missing_CustomerId.copy()
df_no_duplicate.drop_duplicates(inplace=True)
df_no_duplicate.head()

,InvoiceNumber,ProductCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### Step 3 – Filter out invalid prices
Removed rows with non-positive `UnitPrice` values and kept only those with positive prices


In [7]:
df_no_missing_price = df_no_duplicate.copy()
df['CustomerId'] = df['CustomerId'].astype('category')
df_no_missing_price = df_no_missing_price[df_no_missing_price['UnitPrice']>0]
df_no_missing_price.head()

,InvoiceNumber,ProductCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### Step 4 – Calculate the percentage of cancelled orders
Calculated the percentage of cancelled orders by considering only unique `InvoiceNumber`s starting with 'C' and rounded the result.


In [8]:
df_no_missing_price_copy=df_no_missing_price.copy()
df_no_missing_price_copy=df_no_missing_price_copy.drop_duplicates(subset=['InvoiceNumber'])
a=df_no_missing_price_copy.InvoiceNumber.str.startswith('C').count()
print(a)
print(len(df_no_missing_price_copy))
percentage_cancelled_orders=round((a/len(df_no_missing_price_copy))*100)
percentage_cancelled_orders

4372
23585


19

### Step 5 – Identify top customers with the most cancelled orders
Identified the customers with the highest number of unique cancelled orders


In [ ]:

df_no_missing_price2 = df_no_missing_price.copy()
df_no_missing_price2['CustomerId'] = df_no_missing_price2['CustomerId'].astype('str')
df_no_missing_price2['InvoiceNumber'] = df_no_missing_price2['InvoiceNumber'].astype('str')


cancel_counts = df_no_missing_price2.drop_duplicates(subset=['InvoiceNumber']) \
    .groupby('CustomerId')['InvoiceNumber'] \
    .apply(lambda x: x.str.startswith('C').sum()) \
    .sort_values(ascending=False)

top_cancelling_customers_df = cancel_counts.reset_index()
top_cancelling_customers_df.columns = ['CustomerId', 'CancelledOrdersCount']

df_no_missing_price['CustomerId'] = df_no_missing_price['CustomerId'].astype('category')

top_cancelling_customers_df


,CustomerId,CancelledOrdersCount
0,14911.0,65
1,14527.0,39
2,15311.0,37
3,14156.0,36
4,17841.0,35
...,...,...
4376,12371.0,0
4377,12370.0,0
4378,12369.0,0
4379,12368.0,0


### Step 6 – Remove Canceled Orders

Filtered out the rows with canceled orders from the dataset and stored the result in the new dataframe


In [11]:
df_no_canceled_invoice = df_no_missing_price.copy()

df_no_canceled_invoice = df_no_canceled_invoice[df_no_canceled_invoice['InvoiceNumber'].str.startswith('C')!=True]

df_no_canceled_invoice.head()


,InvoiceNumber,ProductCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [12]:
df_no_canceled_invoice.to_csv("preprocessed_sales.csv", index=False)

### Step 7 – Extract Remaining Unique Invoices

Extracted the unique `InvoiceNumber` values from the dataset of non-canceled orders and stored them in the `remaining_invoices` series.


In [17]:
df_no_canceled_invoice.drop_duplicates(subset=['InvoiceNumber'],inplace=True)
remaining_invoices = df_no_canceled_invoice['InvoiceNumber'].to_frame()
remaining_invoices


,InvoiceNumber
0,489434
8,489435
12,489436
31,489437
54,489438
...,...
525303,538167
525328,538168
525358,538169
525371,538170
